In [17]:
import pandas as pd
import BeautifulSoup as bs
import os
import graphlab as gl
from scipy.sparse import csr_matrix
from scipy import io
import numpy as np
import math
from numpy import inf

In [68]:
data = gl.SFrame({'filename':[""], 'text':[""]})
for root, dirs, files in os.walk('/Users/shiv/Desktop/2012_complete/', topdown=False):
    for idx,name in enumerate(files):
        if "html" in name:
            soup = bs.BeautifulSoup(open(os.path.join(root, name)))
            for elem in soup.findAll(['script', 'style']):
                elem.extract()
            res = soup.find("div", {"id": "content_document"})
            df = gl.SFrame({'filename':[name], 'text':[str(res.text.encode('utf-8'))]})
            data = data.append(df)
docs = gl.text_analytics.count_words(data['text'])
docs = docs.dict_trim_by_keys(gl.text_analytics.stopwords(), exclude=True)
data['tf_idf'] = gl.text_analytics.tf_idf(docs)

1
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276


In [2]:
df1 = gl.load_sframe('/Users/shiv/Desktop/2013_sframe/')
df2 = gl.load_sframe('/Users/shiv/Desktop/2012_sframe/')
df3 = gl.load_sframe('/Users/shiv/Desktop/2011_sframe/')
df1 = df1[1:]
df2 = df2[1:]
df3 = df3[1:]

This non-commercial license of GraphLab Create for academic use is assigned to sp4658@nyu.edu and will expire on March 06, 2018.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1491680827.log


In [3]:
df1 = df1.append(df2)
df1 = df1.append(df3)

In [81]:
model = gl.nearest_neighbors.create(df1, label='filename', features=['tf_idf'],
                                          method='brute_force', distance='euclidean')

Starting brute force nearest neighbors model training.

In [25]:
df1 = df1.add_row_number()

In [27]:
df1[df1['filename']=='X1HVA7GN.html']

id,filename,text,tf_idf
221,X1HVA7GN.html,"PrimaryFederalReporter,ReporterOfDecisions,Blo ...","{'limited':3.1465930634394157, ..."


In [82]:
model.query(df1[df1['filename']=='X1HVA7GN.html'], label='filename', k=10)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 3       | 0.0221943   | 220.113ms    |

| 0            | 12815   | 94.8065     | 1.15s        |

| Done         |         | 100         | 1.27s        |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
X1HVA7GN.html,X1HVA7GN.html,0.0,1
X1HVA7GN.html,X1OM100N.html,1426.44147473,2
X1HVA7GN.html,X1I595I003.html,1762.67085446,3
X1HVA7GN.html,X1JI9AQ003.html,1797.70050359,4
X1HVA7GN.html,X1ETG0Q003.html,1798.10452335,5
X1HVA7GN.html,X1H7814003.html,1798.21828383,6
X1HVA7GN.html,X1FBL2S003.html,1798.67897414,7
X1HVA7GN.html,X1JQS44003.html,1801.75232124,8
X1HVA7GN.html,X1ITAPK003.html,1801.85183069,9
X1HVA7GN.html,X1OJ9LON.html,1801.97230222,10


In [5]:
def dataframe_to_scipy(x, column_name):
    '''
    Convert a dictionary column of an SFrame into a sparse matrix format where
    each (row_id, column_id, value) triple corresponds to the value of
    x[row_id][column_id], where column_id is a key in the dictionary.
       
    Example
    >>> sparse_matrix, map_key_to_index = sframe_to_scipy(sframe, column_name)
    '''
    assert x[column_name].dtype() == dict, \
        'The chosen column must be dict type, representing sparse data.'
        
    # Create triples of (row_id, feature_id, count).
    # 1. Add a row number.
    x = x.add_row_number()
    # 2. Stack will transform x to have a row for each unique (row, key) pair.
    x = x.stack(column_name, ['feature', 'value'])

    # Map words into integers using a OneHotEncoder feature transformation.
    f = gl.feature_engineering.OneHotEncoder(features=['feature'])
    # 1. Fit the transformer using the above data.
    f.fit(x)
    # 2. The transform takes 'feature' column and adds a new column 'feature_encoding'.
    x = f.transform(x)
    # 3. Get the feature mapping.
    mapping = f['feature_encoding']
    # 4. Get the feature id to use for each key.
    x['feature_id'] = x['encoded_features'].dict_keys().apply(lambda x: x[0])

    # Create numpy arrays that contain the data for the sparse matrix.
    i = np.array(x['id'])
    j = np.array(x['feature_id'])
    v = np.array(x['value'])
    
    width = x['id'].max() + 1
    height = x['feature_id'].max() + 1
    # Create a sparse matrix.
    mat = csr_matrix((v, (i, j)), shape=(width, height))

    return mat, mapping

In [6]:
tf_idf, map_index_to_word = dataframe_to_scipy(df1, 'tf_idf')

In [8]:
from sklearn.preprocessing import normalize
tf_idf = normalize(tf_idf)

In [18]:
io.mmwrite("/Users/shiv/Desktop/tf-idf-sparse", tf_idf)

In [ ]:
# back to sparse matrix
newm = io.mmread("/Users/shiv/Desktop/tf-idf-sparse.mtx")
newm.tocsr()  

In [9]:
from sklearn.neighbors import NearestNeighbors
neigh = NearestNeighbors(n_neighbors=10)
neigh.fit(tf_idf) 

NearestNeighbors(algorithm='auto', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=1, n_neighbors=10, p=2, radius=1.0)

In [55]:
neighs = neigh.kneighbors(tf_idf[221], 10, return_distance=False)

In [57]:
neighs = neighs[0]

In [58]:
df1.filter_by(neighs, 'id')

id,filename,text,tf_idf
221,X1HVA7GN.html,"PrimaryFederalReporter,ReporterOfDecisions,Blo ...","{'limited':3.1465930634394157, ..."
2546,X1JUEDO003.html,"PrimaryFederalReporter,ReporterOfDecisions,Blo ...","{'""irreparable':6.360808894147377, ..."
3965,X1OM100N.html,"PrimaryFederalReporter,ReporterOfDecisions,Blo ...","{'limited':2.5172744507515326, ..."
7337,X1I595I003.html,"PrimaryFederalReporter,ReporterOfDecisions,Blo ...","{'limited':1.244764280592849, ..."
8584,X1Q6M6PERF82.html,"Bloomberg,676 F.3d86909-173112012 BL 90 ...","{'considered,':3.262649391414396, ..."
9496,X1F0S9I003.html,"PrimaryFederalReporter,Bloomberg,636 F.3d ...","{'limited':1.3442047497636258, ..."
10348,X1FDIJO003.html,"PrimaryFederalReporter,ReporterOfDecisions,Blo ...","{'."").for':7.098995281283338, ..."
11689,X1G18QE003.html,"PrimaryFederalReporter,Bloomberg,653 F.3d ...","{'limited':1.3442047497636258, ..."
11792,X1G2CE8003.html,"PrimaryFederalReporter,ReporterOfDecisions,Blo ...","{'party[.]""[fn34]indeed,': 8.48528964240323, ..."
12344,X1GE0V6003.html,"PrimaryFederalReporter,ReporterOfDecisions,Blo ...","{'tenn.':4.407752198497509, ..."


In [34]:
soup = bs.BeautifulSoup(open("/Users/shiv/Desktop/2013_complete/X1HVA7GN.html"))
for elem in soup.findAll(['script', 'style']):
    elem.extract()

In [36]:
res = soup.find("div", {"id": "content_document"})

In [37]:
res.text

u'PrimaryFederalReporter,ReporterOfDecisions,Bloomberg,712 F.3d 676106 U.S.P.Q.2d 134112-2786-cv2013 BL 87728Pagination*F.3d**U.S.P.Q.2d***BLMajority Opinion&gt;Dissenting Opinion&gt;United States Court of Appeals for the Second CircuitWNET, Thirteen, Fox Television Stations, Inc., Twentieth Century Fox Film Corporation, WPIX, Inc., Univision Television Group, Inc., The Univision Network Limited Partnership, and Public Broadcasting Service,  Plaintiffs-Counter-Defendants-Appellants,v.Aereo, Inc., f/k/a Bamboom Labs, Inc.,  Defendant-Counter-Claimant-Appellee.American Broadcasting Companies, Inc., Disney Enterprises, Inc., CBS Broadcasting Inc., CBS Studios Inc., NBCUniversal Media, LLC, NBC Studios, LLC, Universal Network Television, LLC, Telemundo Network Group LLC, and WNJU-TV Broadcasting LLC,  Plaintiffs-Counter-Defendants-Appellants,v.Aereo, Inc.,  Defendant-Counter-Claimant-Appellee.Docket Nos.  12-2786-cv,  12-2807-cv.Argued: Nov. 30, 2012.Decided: April 1, 2013[**1342].Hide Hea

In [24]:
pd.options.display.max_columns = 200

In [28]:
merged[merged['caseId']=='2010-001']

,caseId,docketId,caseIssuesId,voteId,dateDecision,decisionType,usCite,sctCite,ledCite,lexisCite,term,naturalCourt,chief,docket,caseName,dateArgument,dateRearg,petitioner,petitionerState,respondent,respondentState,jurisdiction,adminAction,adminActionState,threeJudgeFdc,caseOrigin,caseOriginState,caseSource,caseSourceState,lcDisagreement,certReason,lcDisposition,lcDispositionDirection,declarationUncon,caseDisposition,caseDispositionUnusual,partyWinning,precedentAlteration,voteUnclear,issue,issueArea,decisionDirection,decisionDirectionDissent,authorityDecision1,authorityDecision2,lawType,lawSupp,lawMinor,majOpinWriter,majOpinAssigner,splitVote,majVotes,minVotes,justice,justiceName,vote,opinion,direction,majority,firstAgreement,secondAgreement
73859,2010-001,2010-001-01,2010-001-01-01,2010-001-01-01-01-01,11/8/2010,2,NaN,131 S. Ct. 13,178 L. Ed. 2d 276,2010 U.S. LEXIS 8663,2010,1704,Roberts,10-91,"BILL K. WILSON, SUPERINTENDENT, INDIANA STATE ...",NaN,NaN,28,18.0,215,NaN,1,NaN,NaN,0.0,68.0,NaN,27.0,NaN,0.0,12.0,6.0,2.0,1,5.0,0,1,0,0,10020.0,1.0,1.0,0.0,4.0,NaN,3.0,341.0,NaN,NaN,111.0,1,9,0,111,JGRoberts,1.0,1.0,1.0,2.0,NaN,NaN
73860,2010-001,2010-001-01,2010-001-01-01,2010-001-01-01-01-02,11/8/2010,2,NaN,131 S. Ct. 13,178 L. Ed. 2d 276,2010 U.S. LEXIS 8663,2010,1704,Roberts,10-91,"BILL K. WILSON, SUPERINTENDENT, INDIANA STATE ...",NaN,NaN,28,18.0,215,NaN,1,NaN,NaN,0.0,68.0,NaN,27.0,NaN,0.0,12.0,6.0,2.0,1,5.0,0,1,0,0,10020.0,1.0,1.0,0.0,4.0,NaN,3.0,341.0,NaN,NaN,111.0,1,9,0,105,AScalia,1.0,1.0,1.0,2.0,NaN,NaN
73861,2010-001,2010-001-01,2010-001-01-01,2010-001-01-01-01-03,11/8/2010,2,NaN,131 S. Ct. 13,178 L. Ed. 2d 276,2010 U.S. LEXIS 8663,2010,1704,Roberts,10-91,"BILL K. WILSON, SUPERINTENDENT, INDIANA STATE ...",NaN,NaN,28,18.0,215,NaN,1,NaN,NaN,0.0,68.0,NaN,27.0,NaN,0.0,12.0,6.0,2.0,1,5.0,0,1,0,0,10020.0,1.0,1.0,0.0,4.0,NaN,3.0,341.0,NaN,NaN,111.0,1,9,0,106,AMKennedy,1.0,1.0,1.0,2.0,NaN,NaN
73862,2010-001,2010-001-01,2010-001-01-01,2010-001-01-01-01-04,11/8/2010,2,NaN,131 S. Ct. 13,178 L. Ed. 2d 276,2010 U.S. LEXIS 8663,2010,1704,Roberts,10-91,"BILL K. WILSON, SUPERINTENDENT, INDIANA STATE ...",NaN,NaN,28,18.0,215,NaN,1,NaN,NaN,0.0,68.0,NaN,27.0,NaN,0.0,12.0,6.0,2.0,1,5.0,0,1,0,0,10020.0,1.0,1.0,0.0,4.0,NaN,3.0,341.0,NaN,NaN,111.0,1,9,0,108,CThomas,1.0,1.0,1.0,2.0,NaN,NaN
73863,2010-001,2010-001-01,2010-001-01-01,2010-001-01-01-01-05,11/8/2010,2,NaN,131 S. Ct. 13,178 L. Ed. 2d 276,2010 U.S. LEXIS 8663,2010,1704,Roberts,10-91,"BILL K. WILSON, SUPERINTENDENT, INDIANA STATE ...",NaN,NaN,28,18.0,215,NaN,1,NaN,NaN,0.0,68.0,NaN,27.0,NaN,0.0,12.0,6.0,2.0,1,5.0,0,1,0,0,10020.0,1.0,1.0,0.0,4.0,NaN,3.0,341.0,NaN,NaN,111.0,1,9,0,109,RBGinsburg,1.0,1.0,1.0,2.0,NaN,NaN
73864,2010-001,2010-001-01,2010-001-01-01,2010-001-01-01-01-06,11/8/2010,2,NaN,131 S. Ct. 13,178 L. Ed. 2d 276,2010 U.S. LEXIS 8663,2010,1704,Roberts,10-91,"BILL K. WILSON, SUPERINTENDENT, INDIANA STATE ...",NaN,NaN,28,18.0,215,NaN,1,NaN,NaN,0.0,68.0,NaN,27.0,NaN,0.0,12.0,6.0,2.0,1,5.0,0,1,0,0,10020.0,1.0,1.0,0.0,4.0,NaN,3.0,341.0,NaN,NaN,111.0,1,9,0,110,SGBreyer,1.0,1.0,1.0,2.0,NaN,NaN
73865,2010-001,2010-001-01,2010-001-01-01,2010-001-01-01-01-07,11/8/2010,2,NaN,131 S. Ct. 13,178 L. Ed. 2d 276,2010 U.S. LEXIS 8663,2010,1704,Roberts,10-91,"BILL K. WILSON, SUPERINTENDENT, INDIANA STATE ...",NaN,NaN,28,18.0,215,NaN,1,NaN,NaN,0.0,68.0,NaN,27.0,NaN,0.0,12.0,6.0,2.0,1,5.0,0,1,0,0,10020.0,1.0,1.0,0.0,4.0,NaN,3.0,341.0,NaN,NaN,111.0,1,9,0,112,SAAlito,1.0,1.0,1.0,2.0,NaN,NaN
73866,2010-001,2010-001-01,2010-001-01-01,2010-001-01-01-01-08,11/8/2010,2,NaN,131 S. Ct. 13,178 L. Ed. 2d 276,2010 U.S. LEXIS 8663,2010,1704,Roberts,10-91,"BILL K. WILSON, SUPERINTENDENT, INDIANA STATE ...",NaN,NaN,28,18.0,215,NaN,1,NaN,NaN,0.0,68.0,NaN,27.0,NaN,0.0,12.0,6.0,2.0,1,5.0,0,1,0,0,10020.0,1.0,1.0,0.0,4.0,NaN,3.0,341.0,NaN,NaN,111.0,1,9,0,113,SSotomayor,1.0,1.0,1.0,2.0,NaN,NaN
73867,2010-001,2010-001-01,2010-001-01-01,2010-001-01-01-01-09,11/8/2010,2,NaN,131 S. Ct. 13,178 L. Ed. 2d 276,2010 U.S. LEXIS 

In [ ]:
merged.head()

In [18]:
merged_scotus_judges = pd.read_csv('/Volumes/Seagate Backup Plus Drive/Dropbox (Daniel Chen)/Oyez_vowels/merged_scotus_judges.csv')

In [19]:
predictions = pd.read_csv('/Volumes/Seagate Backup Plus Drive/Dropbox (Daniel Chen)/Oyez_vowels/predictions.csv')


In [20]:
vowels = pd.read_csv('/Volumes/Seagate Backup Plus Drive/Dropbox (Daniel Chen)/Oyez_vowels/vowels.csv')

In [38]:
merged_scotus_judges.head()

,SpeakerId,docket_id,matched_name,caseId,docketId,caseIssuesId,voteId,dateDecision,decisionType,usCite,sctCite,ledCite,lexisCite,term,naturalCourt,chief,docket,caseName,dateArgument,dateRearg,petitioner,petitionerState,respondent,respondentState,jurisdiction,adminAction,adminActionState,threeJudgeFdc,caseOrigin,caseOriginState,caseSource,caseSourceState,lcDisagreement,certReason,lcDisposition,lcDispositionDirection,declarationUncon,caseDisposition,caseDispositionUnusual,partyWinning,precedentAlteration,voteUnclear,issue,issueArea,decisionDirection,decisionDirectionDissent,authorityDecision1,authorityDecision2,lawType,lawSupp,lawMinor,majOpinWriter,majOpinAssigner,splitVote,majVotes,minVotes,justice,justiceName,vote,opinion,direction,majority,firstAgreement,secondAgreement
0,1.0,1998_96_1570,David H. Souter,1998-008,1998-008-01,1998-008-01-01,1998-008-01-01-01-06,12/14/1998,1,525 U.S. 128,119 S. Ct. 493,142 L. Ed. 2d 510,1998 U.S. LEXIS 8080,1998,1607,Rehnquist,96-1570,"NYNEX CORPORATION, et al. v. DISCON, INCORPORATED",10/5/1998,NaN,122.0,NaN,233.0,NaN,1,NaN,NaN,0,NaN,NaN,22.0,NaN,0,7.0,2.0,2.0,1.0,5.0,0.0,1.0,0.0,0.0,80010.0,8.0,1.0,0,4.0,NaN,3.0,367.0,NaN,110.0,102.0,1,9,0,107,DHSouter,1.0,1.0,1.0,2.0,NaN,NaN
1,2.0,1998_96_1570,Antonin Scalia,1998-008,1998-008-01,1998-008-01-01,1998-008-01-01-01-04,12/14/1998,1,525 U.S. 128,119 S. Ct. 493,142 L. Ed. 2d 510,1998 U.S. LEXIS 8080,1998,1607,Rehnquist,96-1570,"NYNEX CORPORATION, et al. v. DISCON, INCORPORATED",10/5/1998,NaN,122.0,NaN,233.0,NaN,1,NaN,NaN,0,NaN,NaN,22.0,NaN,0,7.0,2.0,2.0,1.0,5.0,0.0,1.0,0.0,0.0,80010.0,8.0,1.0,0,4.0,NaN,3.0,367.0,NaN,110.0,102.0,1,9,0,105,AScalia,1.0,1.0,1.0,2.0,NaN,NaN
2,5.0,1998_96_1570,John Paul Stevens,1998-008,1998-008-01,1998-008-01-01,1998-008-01-01-01-02,12/14/1998,1,525 U.S. 128,119 S. Ct. 493,142 L. Ed. 2d 510,1998 U.S. LEXIS 8080,1998,1607,Rehnquist,96-1570,"NYNEX CORPORATION, et al. v. DISCON, INCORPORATED",10/5/1998,NaN,122.0,NaN,233.0,NaN,1,NaN,NaN,0,NaN,NaN,22.0,NaN,0,7.0,2.0,2.0,1.0,5.0,0.0,1.0,0.0,0.0,80010.0,8.0,1.0,0,4.0,NaN,3.0,367.0,NaN,110.0,102.0,1,9,0,103,JPStevens,1.0,1.0,1.0,2.0,NaN,NaN
3,6.0,1998_96_1570,Anthony M. Kennedy,1998-008,1998-008-01,1998-008-01-01,1998-008-01-01-01-05,12/14/1998,1,525 U.S. 128,119 S. Ct. 493,142 L. Ed. 2d 510,1998 U.S. LEXIS 8080,1998,1607,Rehnquist,96-1570,"NYNEX CORPORATION, et al. v. DISCON, INCORPORATED",10/5/1998,NaN,122.0,NaN,233.0,NaN,1,NaN,NaN,0,NaN,NaN,22.0,NaN,0,7.0,2.0,2.0,1.0,5.0,0.0,1.0,0.0,0.0,80010.0,8.0,1.0,0,4.0,NaN,3.0,367.0,NaN,110.0,102.0,1,9,0,106,AMKennedy,1.0,1.0,1.0,2.0,NaN,NaN
4,8.0,1998_96_1570,Ruth Bader Ginsburg,1998-008,1998-008-01,1998-008-01-01,1998-008-01-01-01-08,12/14/1998,1,525 U.S. 128,119 S. Ct. 493,142 L. Ed. 2d 510,1998 U.S. LEXIS 8080,1998,1607,Rehnquist,96-1570,"NYNEX CORPORATION, et al. v. DISCON, INCORPORATED",10/5/1998,NaN,122.0,NaN,233.0,NaN,1,NaN,NaN,0,NaN,NaN,22.0,NaN,0,7.0,2.0,2.0,1.0,5.0,0.0,1.0,0.0,0.0,80010.0,8.0,1.0,0,4.0,NaN,3.0,367.0,NaN,110.0,102.0,1,9,0,109,RBGinsburg,1.0,1.0,1.0,2.0,NaN,NaN


In [39]:
predictions.head()

,docketId,docket_id,justice,SpeakerId,admin_action,case_origin,case_origin_circuit,case_source,case_source_circuit,cert_reason,court_direction_circuit_origin_mean,court_direction_circuit_origin_mean_10,court_direction_circuit_source_mean,court_direction_circuit_source_mean_10,court_direction_issue_mean,court_direction_issue_mean_10,court_direction_mean,court_direction_mean_10,court_direction_petitioner_mean,court_direction_petitioner_mean_10,court_direction_respondent_mean,court_direction_respondent_mean_10,current_court_agree_mean,current_court_agree_std,current_court_direction_circuit_origin_mean,current_court_direction_circuit_origin_std,current_court_direction_circuit_source_mean,current_court_direction_circuit_source_std,current_court_direction_issue_mean,current_court_direction_issue_mean_z,current_court_direction_issue_std,current_court_direction_mean,current_court_direction_petitioner_mean,current_court_direction_petitioner_std,current_court_direction_respondent_mean,current_court_direction_respondent_std,current_court_direction_std,diff_court_lc_direction,diff_court_lc_direction_abs,diff_court_lc_direction_abs_z,diff_court_lc_direction_z,diff_justice_court_direction,diff_justice_court_direction_abs,diff_justice_court_direction_issue,diff_justice_court_direction_issue_abs,diff_justice_court_direction_issue_z,diff_justice_court_direction_petitioner,diff_justice_court_direction_petitioner_abs,diff_justice_court_direction_respondent,diff_justice_court_direction_respondent_abs,...,issue,issue_area,jurisdiction,justice_agree_mean,justice_agree_mean_10,justice_direction_circuit_origin_mean,justice_direction_circuit_origin_mean_10,justice_direction_circuit_source_mean,justice_direction_circuit_source_mean_10,justice_direction_issue_mean,justice_direction_issue_mean_10,justice_direction_issue_mean_z,justice_direction_mean,justice_direction_mean_10,justice_direction_mean_z,justice_direction_petitioner_mean,justice_direction_petitioner_mean_10,justice_direction_respondent_mean,justice_direction_respondent_mean_10,law_type,lc_direction,lc_disagreement,lc_disposition,lower_court_direction_circuit_source_mean,lower_court_direction_circuit_source_mean_10,lower_court_direction_issue_mean,lower_court_direction_issue_mean_10,lower_court_direction_petitioner_mean,lower_court_direction_petitioner_mean_10,lower_court_direction_respondent_mean,lower_court_direction_respondent_mean_10,month_argument,month_decision,natural_court,outcome,party_president,petitioner,petitioner_dk,respondent,respondent_dk,segal_cover,year_of_birth,prediction,prob_prediction0,prob_prediction1,target,docket_vote_mean,docket_vote_sum,correct,year
0,1998-008-01,1998_96_1570,107,1.0,-99.0,-99.0,-99.0,22.0,2.0,7.0,-99.0,-99.0,1.507752,1.1,1.680693,1.5,1.521238,1.7,1.526316,1.4,1.647059,1.7,0.802987,0.044931,-99.0,-99.0,1.411376,0.111761,1.452491,0.176781,0.147878,1.435138,1.22619,0.178174,1.457937,0.206663,0.110706,-0.564862,0.564862,0.906323,-98.0,0.085207,0.085207,0.047509,0.047509,0.321275,0.107143,0.107143,0.042063,0.042063,...,80010.0,8.0,1,0.830588,0.9,-99.0,-99.0,1.487179,1.2,1.500000,1.5,0.357812,1.520345,1.8,0.074195,1.333333,-99.0,1.500000,-99.0,3.0,2.0,0,2.0,1.47907,1.7,1.371287,1.6,1.631579,1.6,1.235294,1.1,10.0,12,1607,1,1,122.0,1.0,233.0,1.0,0.325,1939,0,0.662674,0.337326,1,0,0,False,1998
1,1998-008-01,1998_96_1570,105,2.0,-99.0,-99.0,-99.0,22.0,2.0,7.0,-99.0,-99.0,1.507752,1.1,1.680693,1.5,1.521238,1.7,1.526316,1.4,1.647059,1.7,0.802987,0.044931,-99.0,-99.0,1.411376,0.111761,1.452491,0.176821,0.147878,1.435138,1.22619,0.178174,1.457937,0.206663,0.110706,-0.564862,0.564862,0.906689,-98.0,-0.083537,0.083537,-0.008046,0.008046,-0.054411,-0.083333,0.083333,0.042063,0.042063,...,80010.0,8.0,1,0.820178,0.8,-99.0,-99.0,1.421053,-99.0,1.444444,1.3,0.356150,1.351601,1.5,-0.038430,1.142857,-99.0,1.500000,-99.0,3.0,2.0,0,2.0,1.47907,1.7,1.371287,1.6,1.631579,1.6,1.235294,1.1,10.0,12,1607,1,1,122.0,1.0,233.0,1.0,0.000,1936,0,0.633697,0.366303,1,0,0,False,1998
2,199

In [42]:
vowels

,stress,vowel,vowel_id
0,0.0,AW,0
1,1.0,AY,1
2,0.0,AA,2
3,1.0,OY,3
4,2.0,ER,4
5,2.0,UH,5
6,2.0,EY,6
7,1.0,AE,7
8,0.0,IY,8
9,0.0,EH,9
